In [1]:
from deconvtest import Step
from deconvtest import Workflow

import numpy as np

In [2]:
w = Workflow(name='test workflow')
w.steps

[]

In [3]:
w.available_steps

[{
     "name": "Convolution",
     "number of inputs": 2,
     "number of outputs": 1,
     "available methods": [
         "convolve"
     ]
 },
 {
     "name": "Deconvolution",
     "number of inputs": 1,
     "number of outputs": 1,
     "available methods": [
         "regularized_inverse_filter"
     ]
 },
 {
     "name": "Evaluation",
     "number of inputs": 2,
     "number of outputs": 0,
     "available methods": [
         "rmse"
     ]
 },
 {
     "name": "GroundTruth",
     "number of inputs": 0,
     "number of outputs": 1,
     "available methods": [
         "ellipsoid"
     ]
 },
 {
     "name": "PSF",
     "number of inputs": 0,
     "number of outputs": 1,
     "available methods": [
         "gaussian"
     ]
 },
 {
     "name": "Transform",
     "number of inputs": 1,
     "number of outputs": 1,
     "available methods": [
         "poisson_noise"
     ]
 }]

In [4]:
s = Step('GroundTruth')
s

{
    "name": "GroundTruth",
    "number of inputs": 0,
    "number of outputs": 1,
    "available methods": [
        "ellipsoid"
    ]
}

In [5]:
s.valid_parameters

In [6]:
s = Step('GroundTruth', 'ellipsoid')
s

{
    "name": "GroundTruth",
    "number of inputs": 0,
    "number of outputs": 1,
    "method": "ellipsoid",
    "parameter_path": null,
    "number of parameter combinations": 0,
    "input step": null,
    "valid parameters": [
        "Parameter: name=size, value=None, type=typing.Union[int, float, list, numpy.ndarray], optional=False, default_value=None",
        "Parameter: name=voxel_size, value=None, type=typing.Union[int, float, list, numpy.ndarray], optional=True, default_value=1.0",
        "Parameter: name=theta, value=None, type=typing.Union[int, float], optional=True, default_value=0",
        "Parameter: name=phi, value=None, type=typing.Union[int, float], optional=True, default_value=0"
    ]
}

In [7]:
s.specify_parameters(size=[[10,6,6], 10], 
                     voxel_size=[[0.5, 0.2, 0.2]],
                     theta=[np.pi / 2, 0], 
                     phi=[np.pi * 4 / 3, 0],
                     mode='align', base_name='GT')
s

/Users/amedyukh/Documents/StJude/DeconvTest/deconvtest2/deconvtest/framework/workflow/step.py:151: UserWarning: Since list is a valid type for parameter size and the number of provided values is <= 3, values [[10, 6, 6], 10] will be assumed as 2 different values.To specify one value for different dimensions, wrap them up in another list, e.g:[[value_dim1, value_dim2, value_dim2]]
  warnings.warn(rf'Since list is a valid type for parameter {param.name} and '
/Users/amedyukh/Documents/StJude/DeconvTest/deconvtest2/deconvtest/framework/workflow/step.py:151: UserWarning: Since list is a valid type for parameter voxel_size and the number of provided values is <= 3, values [[0.5, 0.2, 0.2]] will be assumed as 1 different values.To specify one value for different dimensions, wrap them up in another list, e.g:[[value_dim1, value_dim2, value_dim2]]
  warnings.warn(rf'Since list is a valid type for parameter {param.name} and '


{
    "name": "GroundTruth",
    "number of inputs": 0,
    "number of outputs": 1,
    "method": "ellipsoid",
    "parameter_path": null,
    "number of parameter combinations": 2,
    "input step": null
}

In [8]:
s.parameters

,size,theta,phi,voxel_size_0,voxel_size_1,voxel_size_2,ID
0,"[10, 6, 6]",1.570796,4.18879,0.5,0.2,0.2,GT0000
1,10,0.000000,0.00000,0.5,0.2,0.2,GT0001


In [9]:
s.save_parameters('params_ellipsoid.csv')
s

{
    "name": "GroundTruth",
    "number of inputs": 0,
    "number of outputs": 1,
    "method": "ellipsoid",
    "parameter_path": "params_ellipsoid.csv",
    "number of parameter combinations": 2,
    "input step": null
}

In [10]:
w.add_step(s)
w.steps

[{
     "name": "GroundTruth",
     "number of inputs": 0,
     "number of outputs": 1,
     "method": "ellipsoid",
     "parameter_path": "params_ellipsoid.csv",
     "number of parameter combinations": 2,
     "input step": []
 }]

In [11]:
s = Step('PSF', 'gaussian')
s.specify_parameters(sigma=[1, 2], aspect=[3, 2, 4], mode='permute')
s.save_parameters('params_psf.csv')
w.add_step(s)
s.parameters

,sigma,aspect,voxel_size,ID
0,1,3,1.0,PSF0000
1,1,2,1.0,PSF0001
2,1,4,1.0,PSF0002
3,2,3,1.0,PSF0003
4,2,2,1.0,PSF0004
5,2,4,1.0,PSF0005


In [12]:
s = Step('Convolution', 'convolve')
s.specify_parameters(img='pipeline', psf='pipeline')
w.add_step(s, input_step=[0, 1])
s

{
    "name": "Convolution",
    "number of inputs": 2,
    "number of outputs": 1,
    "method": "convolve",
    "parameter_path": null,
    "number of parameter combinations": 1,
    "input step": [
        0,
        1
    ]
}

In [13]:
s.parameters

,ID
0,Convolution0000


In [14]:
s = Step('Transform', 'poisson_noise')
s.specify_parameters(img='pipeline', snr=[2, 5, 10], base_name='noise')
s.save_parameters('params_noise.csv')
w.add_step(s)
s.parameters

,snr,ID
0,2,noise0000
1,5,noise0001
2,10,noise0002


In [15]:
s = Step('Evaluation', 'rmse')
s.specify_parameters(img1='pipeline', img2='pipeline')
w.add_step(s, input_step=[0, 3])
s

{
    "name": "Evaluation",
    "number of inputs": 2,
    "number of outputs": 0,
    "method": "rmse",
    "parameter_path": null,
    "number of parameter combinations": 1,
    "input step": [
        0,
        3
    ]
}

In [16]:
w.save('workflow.json')
w

{
    "name": "test workflow",
    "path": "workflow.json",
    "steps": [
        {
            "name": "GroundTruth",
            "number of inputs": 0,
            "number of outputs": 1,
            "method": "ellipsoid",
            "parameter_path": "params_ellipsoid.csv",
            "number of parameter combinations": 2,
            "input step": []
        },
        {
            "name": "PSF",
            "number of inputs": 0,
            "number of outputs": 1,
            "method": "gaussian",
            "parameter_path": "params_psf.csv",
            "number of parameter combinations": 6,
            "input step": []
        },
        {
            "name": "Convolution",
            "number of inputs": 2,
            "number of outputs": 1,
            "method": "convolve",
            "parameter_path": null,
            "number of parameter combinations": 1,
            "input step": [
                0,
                1
            ]
        },
        {
           

In [17]:
w2 = Workflow()
w2.load('workflow.json')
w2

{
    "name": "test workflow",
    "path": "workflow.json",
    "steps": [
        {
            "name": "GroundTruth",
            "number of inputs": 0,
            "number of outputs": 1,
            "method": "ellipsoid",
            "parameter_path": "params_ellipsoid.csv",
            "number of parameter combinations": 2,
            "input step": []
        },
        {
            "name": "PSF",
            "number of inputs": 0,
            "number of outputs": 1,
            "method": "gaussian",
            "parameter_path": "params_psf.csv",
            "number of parameter combinations": 6,
            "input step": []
        },
        {
            "name": "Convolution",
            "number of inputs": 2,
            "number of outputs": 1,
            "method": "convolve",
            "parameter_path": null,
            "number of parameter combinations": 0,
            "input step": [
                0,
                1
            ]
        },
        {
           

In [18]:
w.save_workflow_graph('workflow_graph.json')

In [19]:
w.workflow

{'name': 'workflow_graph',
 'items': [{'name': 'item000',
   'steps': [{'name': 'GroundTruth',
     'method': 'ellipsoid',
     'size': [10, 6, 6],
     'theta': 1.5707963267948966,
     'phi': 4.1887902047863905,
     'voxel_size': [0.5, 0.2, 0.2],
     'outputID': 'GT0000'},
    {'name': 'PSF',
     'method': 'gaussian',
     'sigma': 1,
     'aspect': 3,
     'voxel_size': 1.0,
     'outputID': 'PSF0000'},
    {'name': 'Convolution',
     'method': 'convolve',
     'outputID': 'GT0000_PSF0000_Convolution0000'},
    {'name': 'Transform',
     'method': 'poisson_noise',
     'snr': 2,
     'outputID': 'GT0000_PSF0000_Convolution0000_noise0000'}]},
  {'name': 'item001',
   'steps': [{'name': 'GroundTruth',
     'method': 'ellipsoid',
     'size': [10, 6, 6],
     'theta': 1.5707963267948966,
     'phi': 4.1887902047863905,
     'voxel_size': [0.5, 0.2, 0.2],
     'outputID': 'GT0000'},
    {'name': 'PSF',
     'method': 'gaussian',
     'sigma': 1,
     'aspect': 3,
     'voxel_size': 